#Clasificacion de arritmias cardicas en Electrocardigramas (ECG)

* Instalando paquetes necesarios

In [0]:
!pip install wfdb
!pip install numpy

     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 163kB 20.8MB/s 
  Created wheel for wfdb: filename=wfdb-2.2.1-cp36-none-any.whl size=100368 sha256=313cb71811b9ae5fe12a679c4154ba8b12be6ed1f0b7197ef136b80cdeb6fb10
  Stored in directory: /root/.cache/pip/wheels/bb/a9/00/0078d26b0c15b31be0001af8eb659496709c361c69641303f1
Successfully built wfdb


In [ ]:
import numpy as np
import sys, os
import wfdb
import pywt
import matplotlib.pyplot as plt
import pickle as pk
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score 
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam



* ### importando google colab drive
utlilizamos la libreria *drive* para poder utlizar los archivos en drive 
nos pide una autorizacion desde nuestra cuenta

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd gdrive/My\ Drive/'Colab Notebooks'/'ECG_Clasification'

/content/gdrive/My Drive/Colab Notebooks/ECG_Clasification


In [2]:

data_names = ['100', '101', '102', '103', '104', '105', '106', '107', 
              '108', '109', '111', '112', '113', '114', '115', '116', 
              '117', '118', '119', '121', '122', '123', '124', '200', 
              '201', '202', '203', '205', '207', '208', '209', '210', 
              '212', '213', '214', '215', '217', '219', '220', '221', 
              '222', '223', '228', '230', '231', '232', '233', '234']



In [0]:
#tamnoho de las ventanas derecha e izquierdas
widb = 99
wida = 160
n_samples = 650000 #<650000


In [3]:
cN=0
cL=0
cR=0
cA=0
cV=0
labels = ['N', 'L', 'R', 'A', 'V']
X = []
Y = []

for d in data_names:
    r=wfdb.rdrecord('./data/'+d,sampto=n_samples)
    ann=wfdb.rdann('./data/'+d, 'atr', return_label_elements=['label_store', 'symbol'],sampto=n_samples)
    if d!='114': # since 114 is reversed
        sig = np.array(r.p_signal[:,0])
    else:
        sig = np.array(r.p_signal[:,1])
    sig_len = len(sig)
    #print("sig_len ",sig_len)
    sym = ann.symbol
    #print("ann.symbol ",sym)
    #print(sig)
    pos = ann.sample
    #print("ann.sample ",pos)
    beat_len = len(sym)
    print("patient code ",d)
    #print("beat_len ",beat_len)
    
    for i in range(beat_len):
        if sym[i] in labels and pos[i]-widb>=0 and pos[i]+wida+1<=sig_len:
        #if sym[i] in labels and pos[i]-wid>=0 and pos[i]+wid+1<=sig_len:
            a = sig[pos[i]-widb:pos[i]+wida+1]
            #print(a)
            if len(a) != 260:
                print("Length error")
                continue
            X.append(a)
            Y.append(labels.index(sym[i]))
            #print('label',sym[i])
            #print('sym[i] :',sym[i])
            #print('labels.index(sym[i])   ',labels.index(sym[i]))
            if(sym[i]=='N'):
              cN=cN+1
            elif(sym[i]=='L'):
              cL=cL+1
            elif(sym[i]=='R'):
              cR=cR+1
            elif(sym[i]=='A'):
              cA=cA+1
            elif(sym[i]=='V'):
              cV=cV+1
    #wfdb.plot_wfdb(record=r,annotation=ann,title=d+" normal",time_units='seconds')
print('N :',cN)
print('L :',cL)
print('R :',cR)
print('A :',cA)
print('V :',cV)
print('Total NLRAV :',len(X))



patient code  100
patient code  101
patient code  102
patient code  103
patient code  104
patient code  105
patient code  106
patient code  107
patient code  108
patient code  109
patient code  111
patient code  112
patient code  113
patient code  114
patient code  115
patient code  116
patient code  117
patient code  118
patient code  119
patient code  121
patient code  122
patient code  123
patient code  124
patient code  200
patient code  201
patient code  202
patient code  203
patient code  205
patient code  207
patient code  208
patient code  209
patient code  210
patient code  212
patient code  213
patient code  214
patient code  215
patient code  217
patient code  219
patient code  220
patient code  221
patient code  222
patient code  223
patient code  228
patient code  230
patient code  231
patient code  232
patient code  233
patient code  234
N : 75020
L : 8072
R : 7255
A : 2546
V : 7129
Total NLRAV : 100022


In [ ]:
# nsvfq

In [9]:
    labels = ['N', 'S', 'V', 'F', 'Q']
    sub_labels = ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', '/', 'f', 'Q']
    sub = {'N':'N', 'L':'N', 'R':'N', 'e':'N', 'j':'N', 
           'A':'S', 'a':'S', 'J':'S', 'S':'S',
           'V':'V', 'E':'V',
           'F':'F',
           '/':'Q', 'f':'Q', 'Q':'Q'}
    X = []
    Y = []
    for d in data_names:
        r=wfdb.rdrecord('./data/'+d)
        ann=wfdb.rdann('./data/'+d, 'atr', return_label_elements=['label_store', 'symbol'])
        if d!='114':
            sig = np.array(r.p_signal[:,0])
        else:
            sig = np.array(r.p_signal[:,1])
        sig_len = len(sig)
        sym = ann.symbol
        pos = ann.sample
        beat_len = len(sym)
        for i in range(beat_len):
            if sym[i] in labels and pos[i]-widb>=0 and pos[i]+wida+1<=sig_len:
                a = sig[pos[i]-widb:pos[i]+wida+1]
                if len(a) != 260:
                    print("Length error")
                    continue
                X.append(a)
                Y.append(labels.index(sub[sym[i]]))


In [10]:
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)
count = Counter(Y)
print(count)


(82986, 260)
(82986,)
Counter({0: 75020, 2: 7129, 3: 802, 4: 33, 1: 2})


# Cross validation

In [11]:
seed_ = 200 #se puede cambiar para variar los datos y reproducir los resultados
data_len = len(X)
np.random.seed(seed_)
idx = list(range(data_len))


In [12]:
np.random.shuffle(idx)

train_len = int(data_len*0.6) # 60%
valid_len = int(data_len*0.2) # 20%
test_len = data_len-train_len-valid_len # 20%


In [13]:
X_train = X[idx][:train_len]
X_valid = X[idx][train_len:train_len+valid_len]
X_test = X[idx][train_len+valid_len:]
Y_train = Y[idx][:train_len]
Y_valid = Y[idx][train_len:train_len+valid_len]
Y_test = Y[idx][train_len+valid_len:]



print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print(Counter(Y_train))
print(Counter(Y_valid))
print(Counter(Y_test))


(49791, 260)
(16597, 260)
(16598, 260)
Counter({0: 44982, 2: 4298, 3: 490, 4: 20, 1: 1})
Counter({0: 15027, 2: 1395, 3: 167, 4: 8})
Counter({0: 15011, 2: 1436, 3: 145, 4: 5, 1: 1})


Expandiendo dimenciones para utilizar el parametro a modelo neuraonal

In [14]:
X_train = np.expand_dims(X_train, axis=-1)
X_valid = np.expand_dims(X_valid, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)


print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(49791, 260, 1)
(16597, 260, 1)
(16598, 260, 1)


In [15]:
f_size = X_train.shape[1]
class_num = 5

lr = 0.01
batch_size=32

print(Y_train)
Y_train = keras.utils.to_categorical(Y_train, num_classes=class_num)
print(Y_train.shape)

[0 0 2 ... 0 0 0]
(49791, 5)


In [16]:
def make_model(model_type):
    model = Sequential()
    if model_type == '1D':
        model.add(Conv1D(10, 3, activation='relu', input_shape=(f_size,1)))
        model.add(MaxPooling1D(2))
        model.add(Conv1D(10, 3, activation='relu'))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
    elif model_type == '1D-large':
        model.add(Conv1D(50, 13, activation='relu', input_shape=(f_size,1)))
        model.add(MaxPooling1D(2))
        model.add(Conv1D(50, 13, activation='relu'))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
    elif model_type == 'LSTM':
        model.add(LSTM(64, return_sequences=True, dropout=0.1, input_shape=(f_size, 1)))
        model.add(LSTM(32, return_sequences=True, dropout=0.1))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
    elif model_type == 'BiLSTM':
        model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.1), merge_mode='sum', input_shape=(f_size, 1)))
        model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.1), merge_mode='sum'))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
    elif model_type == 'ConvLSTM':
        model = Sequential()
        model.add(Conv1D(10, 7, activation='relu', input_shape=(260,1)))
        model.add(MaxPooling1D(3))
        model.add(Conv1D(10 ,7, activation='relu'))
        model.add(MaxPooling1D(3))
        model.add(LSTM(32, return_sequences=True,recurrent_dropout=0.25))
        model.add(Flatten())
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))


    model.add(Dense(class_num, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])
    return model
 

In [17]:
def make_model_cnn_lstm():
    model = Sequential()
    model.add(Conv1D(10, 7, activation='relu', input_shape=(260,1)))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(10 ,7, activation='relu'))
    model.add(MaxPooling1D(3))
    #model.add(Conv1D(10 ,6, activation='relu'))
    #model.add(MaxPooling1D(2))
    #model.add(LSTM(64, return_sequences=True,recurrent_dropout=0.25))
    model.add(LSTM(32, return_sequences=True,recurrent_dropout=0.25))
    model.add(Flatten())
    #model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr))
    return model
 

In [18]:
import time
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [21]:

model = make_model('1D')
#model = make_model_cnn_lstm()
best_SE = 0
best_ACC = 0
patience = 30
pcnt = 0

best_model = make_model('1D')

bin_label = lambda x: min(1,x)
#model.build()
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 258, 10)           40        
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 129, 10)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 127, 10)           310       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 63, 10)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 630)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               63100     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                

In [0]:
timecallback = TimeHistory()
model.fit(X_train, Y_train, batch_size=batch_size,callbacks=[timecallback], epochs=100, verbose=1)

y_pred = model.predict(X_test)
print('shape predicted',y_pred.shape)

y_pred = np.argmax(y_pred, axis=1)
print('predicted argmax axis = 1',y_pred)
print('predicted argmax shape',y_pred.shape)
correct_predicted = np.sum(y_pred==Y_test)
print('n samples correct predicted : ',correct_predicted)
acc = correct_predicted/len(Y_test)


y_true = list(map(bin_label, Y_test))
y_pred = list(map(bin_label, y_pred))
auc = roc_auc_score(y_true, y_pred)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
SE = tp/(tp+fn)
SP = tn/(fp+tn)
print("Epoch: %d | SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f" %(10, SE, acc, auc, SP))
print("timpo utlizado para entrenar ",sum(timecallback.times))
    
model.save("clasifier.h5")

Epoch 1/100
60013/60013 [==============================] - 8s 138us/step - loss: 0.0492 - acc: 0.9875
Epoch 2/100
60013/60013 [==============================] - 8s 137us/step - loss: 0.0463 - acc: 0.9882
Epoch 3/100
60013/60013 [==============================] - 9s 143us/step - loss: 0.0443 - acc: 0.9887
Epoch 4/100
60013/60013 [==============================] - 8s 140us/step - loss: 0.0398 - acc: 0.9894
Epoch 5/100
60013/60013 [==============================] - 8s 139us/step - loss: 0.0431 - acc: 0.9895
Epoch 6/100
60013/60013 [==============================] - 8s 137us/step - loss: 0.0407 - acc: 0.9896
Epoch 7/100
60013/60013 [==============================] - 8s 136us/step - loss: 0.0365 - acc: 0.9909
Epoch 8/100
60013/60013 [==============================] - 8s 138us/step - loss: 0.0402 - acc: 0.9905
Epoch 9/100
60013/60013 [==============================] - 8s 137us/step - loss: 0.0426 - acc: 0.9903
Epoch 10/100
60013/60013 [==============================] - 8s 137us/step - loss: 

In [ ]:
#entrenamiento y validacion

In [ ]:

save = 'best_train'
for e in range(1, 100+1):

    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1, verbose=0)

    y_pred = model.predict(X_valid)
    y_pred = np.argmax(y_pred, axis=1)
    acc = np.sum(y_pred==Y_valid)/len(Y_valid)

    y_true = list(map(bin_label, Y_valid))
    y_pred = list(map(bin_label, y_pred))
    auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    SE = tp/(tp+fn)
    SP = tn/(fp+tn)

    if SE+acc > best_SE+best_ACC:
        best_SE, best_ACC = SE, acc
        best_model.set_weights(model.get_weights())
        pcnt = 0
    else:
        pcnt += 1
    
    print("Epoch: %d | SE: %.4f | Best SE: %.4f | ACC: %.4f | Best ACC: %.4f | AUC: %.4f | SP: %.4f" %(e, SE, best_SE, acc, best_ACC, auc, SP))
    if pcnt==patience:
        y_pred = best_model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        acc = np.sum(y_pred==Y_test)/len(Y_test)
        y_true = list(map(bin_label, Y_test))
        y_pred = list(map(bin_label, y_pred))
        auc = roc_auc_score(y_true, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        SE = tp/(tp+fn)
        SP = tn/(fp+tn)
        print("CNN Test | SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f" %(SE, acc, auc, SP, best_SE, best_ACC))
        with open("./result/"+save, "a") as fw:
            fw.write("SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f\n" %(SE, acc, auc, SP, best_SE, best_ACC))
        break

model.save('cnn_trained')


Epoch: 1 | SE: 0.9083 | Best SE: 0.9720 | ACC: 0.9887 | Best ACC: 0.9919 | AUC: 0.9533 | SP: 0.9984
Epoch: 2 | SE: 0.9631 | Best SE: 0.9720 | ACC: 0.9922 | Best ACC: 0.9919 | AUC: 0.9801 | SP: 0.9971
Epoch: 3 | SE: 0.9713 | Best SE: 0.9720 | ACC: 0.9921 | Best ACC: 0.9919 | AUC: 0.9837 | SP: 0.9961
Epoch: 4 | SE: 0.9592 | Best SE: 0.9720 | ACC: 0.9908 | Best ACC: 0.9919 | AUC: 0.9778 | SP: 0.9963
Epoch: 5 | SE: 0.9567 | Best SE: 0.9720 | ACC: 0.9924 | Best ACC: 0.9919 | AUC: 0.9770 | SP: 0.9973
Epoch: 6 | SE: 0.9459 | Best SE: 0.9720 | ACC: 0.9908 | Best ACC: 0.9919 | AUC: 0.9715 | SP: 0.9971
Epoch: 7 | SE: 0.9592 | Best SE: 0.9720 | ACC: 0.9921 | Best ACC: 0.9919 | AUC: 0.9783 | SP: 0.9973
Epoch: 8 | SE: 0.9611 | Best SE: 0.9720 | ACC: 0.9928 | Best ACC: 0.9919 | AUC: 0.9797 | SP: 0.9983
Epoch: 9 | SE: 0.9529 | Best SE: 0.9720 | ACC: 0.9913 | Best ACC: 0.9919 | AUC: 0.9749 | SP: 0.9970
Epoch: 10 | SE: 0.9586 | Best SE: 0.9720 | ACC: 0.9916 | Best ACC: 0.9919 | AUC: 0.9778 | SP: 0.9969

In [0]:

import os
import os.path as path

from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
MODEL_NAME = 'clasificador'
if not path.exists('out'):
        os.mkdir('out')
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

In [0]:
  input_node_name = 'input'
  keep_prob_node_name = 'keep_prob'
  output_node_name = 'output'


  
  export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from out/clasificador.chkp
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


AssertionError: ignored

In [0]:
confusion_matrix(y_true, y_pred).ravel()

array([14931,    71,   150,  4853])

In [0]:
bin_label = lambda x: min(1,x)
models = ['1D','1D-large','LSTM','BiLSTM','ConvLSTM']

for mod in models:
  print("Modelo:",mod)
  model = make_model(mod) 
  timecallback = TimeHistory()
  model.fit(X_train, Y_train, batch_size=batch_size,callbacks=[timecallback], epochs=2, verbose=1)

  y_pred = model.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)
  correct_predicted = np.sum(y_pred==Y_test)
  #print('%d ejemplos correctamente clasificados de %d : ',correct_predicted,len(Y_test))
  acc = correct_predicted/len(Y_test)

  y_true = list(map(bin_label, Y_test))
  y_pred = list(map(bin_label, y_pred))
  auc = roc_auc_score(y_true, y_pred)
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
  SE = tp/(tp+fn)
  SP = tn/(fp+tn)
  print("Epoch: %d | ACC: %.4f | SE: %.4f | SP: %.4f | AUC: %.4f " %(2,acc, SE, SP, auc))
  print("tiempo utlizado para entrenar: %.2f segundos"%sum(timecallback.times))

Modelo: 1D






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/2





60013/60013 [==============================] - 15s 252us/step - loss: 0.1665 - acc: 0.9563
Epoch 2/2
60013/60013 [==============================] - 8s 134us/step - loss: 0.0942 - acc: 0.9761
Epoch: 2 | ACC: 0.9749 | SE: 0.9362 | SP: 0.9980 | AUC: 0.9671 
tiempo utlizado para entrenar: 23.18 segundos
Modelo: 1D-large
Epoch 1/2
60013/60013 [==============================] - 9s 148us/step - loss: 0.1592 - acc: 0.9590
Epoch 2/2
60013/60013 [==============================] - 9s 143us/step - loss: 0.1051 - acc: 0.9745
Epoch: 2 | ACC: 0.9798 | SE: 0.9654 | SP: 0.9931 | AUC: 0.9793 
tiempo utlizado para entrenar: 17.43 segundos
Modelo: LSTM

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/2
60013/60013 [==============================] - 1477s 25ms/step - loss: 0.3348 - acc: 0.9067
Epoch 2/2
60

In [0]:
#make convolutional autoencoder
# ENCODER
input_sig = Input(batch_shape=(None,260,1))
x = Conv1D(16,5, activation='relu', padding='valid')(input_sig)
x1 = MaxPooling1D(2)(x)
x2 = Conv1D(64,5, activation='relu', padding='valid')(x1)
x3 = BatchNormalization()(x2)
x4 = MaxPooling1D(2)(x3)
x5 = Conv1D(32,3, activation='relu', padding='valid')(x4)
x6 = Conv1D(1,3, activation='relu', padding='valid')(x5)
encoded = MaxPooling1D(2)(x6)
#encoded = Dense(32,activation = 'relu')(flat)
 
print("shape of encoded {}".format(K.int_shape(encoded)))
 
# DECODER 
x6_ = Conv1D(1,3, activation='relu', padding='valid')(x7)
x5_ = Conv1D(32,3, activation='relu', padding='valid')(x6_)
x4_ = MaxPooling1D(2)(x5_)
x3_ = BatchNormalization()(x4_)
x2_ = Conv1D(64,5, activation='relu', padding='valid')(x3_)
x1_ = MaxPooling1D(2)(x_2)
x_ = Conv1D(16,5, activation='relu', padding='valid')(x_1)
upsamp = UpSampling1D(2)(x_)
flat = Flatten()(upsamp)
decoded = Dense(260,activation = 'sigmoid')(flat)
 
print("shape of decoded {}".format(K.int_shape(decoded)))
 
autoencoder = Model(input_sig, decoded)
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
for e in range(1, 15049+1):

    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1, verbose=0)

    y_pred = model.predict(X_valid)
    y_pred = np.argmax(y_pred, axis=1)
    acc = np.sum(y_pred==Y_valid)/len(Y_valid)

    y_true = list(map(bin_label, Y_valid))
    y_pred = list(map(bin_label, y_pred))
    auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    SE = tp/(tp+fn)
    SP = tn/(fp+tn)

    if SE+acc > best_SE+best_ACC:
        best_SE, best_ACC = SE, acc
        best_model.set_weights(model.get_weights())
        pcnt = 0
    else:
        pcnt += 1
    
    print("Epoch: %d | SE: %.4f | Best SE: %.4f | ACC: %.4f | Best ACC: %.4f | AUC: %.4f | SP: %.4f" %(e, SE, best_SE, acc, best_ACC, auc, SP))

    if pcnt==patience:
        y_pred = best_model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        acc = np.sum(y_pred==Y_test)/len(Y_test)
        y_true = list(map(bin_label, Y_test))
        y_pred = list(map(bin_label, y_pred))
        auc = roc_auc_score(y_true, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        SE = tp/(tp+fn)
        SP = tn/(fp+tn)
        print(mode+" Test | SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f" %(SE, acc, auc, SP, best_SE, best_ACC))
        with open("./result/"+save, "a") as fw:
            fw.write("SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f\n" %(SE, acc, auc, SP, best_SE, best_ACC))
        break



